In [12]:
import pandas as pd
import hvplot.pandas
from pathlib import Path
%matplotlib inline
%store -r dates_FOMC
from datetime import timedelta

In [13]:
dates_FOMC = dates_FOMC.sort_values()
dates_FOMC[48]

Timestamp('2022-05-04 00:00:00')

In [14]:
path = Path('VIX_History.csv')
vix_df = pd.read_csv(path, index_col = 'DATE', parse_dates = True, infer_datetime_format = True)
vix_df.head()

,OPEN,HIGH,LOW,CLOSE
DATE,,,,
1990-01-02,17.24,17.24,17.24,17.24
1990-01-03,18.19,18.19,18.19,18.19
1990-01-04,19.22,19.22,19.22,19.22
1990-01-05,20.11,20.11,20.11,20.11
1990-01-08,20.26,20.26,20.26,20.26


In [15]:
vix_smaller_df = pd.DataFrame(vix_df.loc['2017-01-01':'2022-05-05'].iloc[:,3])

dates_FOMC_df = pd.DataFrame(dates_FOMC)

vix_smaller_df.head()
# for date in dates_FOMC:
#     dates_FOMC.index(
    

,CLOSE
DATE,
2017-01-03,12.85
2017-01-04,11.85
2017-01-05,11.67
2017-01-06,11.32
2017-01-09,11.56


In [16]:
def date_bump_down(date):
    if date.dayofweek == 0:
        date_down = date - timedelta(days = 3)
    else:
        date_down = date - timedelta(days = 1)
    return date_down

def date_bump_up(date):
    if date.dayofweek == 4:
        date_up = date + timedelta(days = 3)
    else:
        date_up = date + timedelta(days = 1)
    return date_up

In [17]:
b4 = []
dayof = []
after = []

for date in dates_FOMC:
    b4.append(vix_smaller_df.loc[date_bump_down(date),'CLOSE'])
    dayof.append(vix_smaller_df.loc[date,'CLOSE'])
    after.append(vix_smaller_df.loc[date_bump_up(date),'CLOSE'])
    
print(len(b4)) 
print(len(dayof))
print(len(after))
d = {'before': b4, 'day of': dayof, 'after': after}

trend = pd.DataFrame(d)
trendy = trend.describe(include='all')
trend_mean = trendy.iloc[1,:]
trend_5 = trendy.iloc[1,:] - 1 * trendy.iloc[2,:]
trend_95 = trendy.iloc[1,:] + 1 * trendy.iloc[2,:]
trendy
#trend_mean.hvplot()
# find a way to normalize the data here
trend_mean.hvplot()*trend_5.hvplot()*trend_95.hvplot()

49
49
49


:Overlay
   .Curve.Mean   :Curve   [index]   (mean)
   .Curve.A_0.I  :Curve   [index]   (0)
   .Curve.A_0.II :Curve   [index]   (0)

In [18]:
vix_change_wide = []
for date in dates_FOMC:
    change = (vix_smaller_df.loc[date_bump_down(date),'CLOSE'] - vix_smaller_df.loc[date_bump_up(date),'CLOSE'])/vix_smaller_df.loc[date_bump_down(date),'CLOSE']
    vix_change_wide.append(change)

vix_change_wide
# vix_change_wide_df = pd.DataFrame(vix_change_wide, )
# vix_change_wide_df

[0.005004170141784862,
 0.08861788617886177,
 0.012275731822473938,
 -0.04606525911708258,
 -0.07211028632025449,
 0.05009823182711196,
 0.024557956777996073,
 -0.05745967741935487,
 0.08924949290060842,
 -0.28241758241758247,
 -0.026468689477081996,
 0.01782820097244738,
 0.04988308651597822,
 0.0008051529790660054,
 -0.06112469437652812,
 -0.10946051602814702,
 0.13382122320961834,
 -0.0051622418879056255,
 -0.0990853658536586,
 0.026402640264026424,
 -0.2819225251076041,
 0.027008310249307395,
 0.06589958158995823,
 -0.0015151515151516175,
 0.11096938775510205,
 0.04852579852579858,
 0.04278874925194504,
 0.05150836516927937,
 0.1361674296926095,
 0.0661720169594186,
 0.0003503854239662933,
 -0.01727733095025315,
 -0.4795067101922379,
 0.02672955974842766,
 0.013321873657069133,
 -0.033997655334114925,
 0.15928305715251947,
 0.04193971166448234,
 -0.31233709817549965,
 -0.09044972208185949,
 -0.0028473804100228196,
 -0.04289071680376031,
 0.08574380165289257,
 0.2352216748768473,
 0

In [19]:
vix_change = []
for date in dates_FOMC:
    change = (vix_df.iloc[:,0].loc[date] - vix_df.iloc[:,3].loc[date])/vix_df.iloc[:,0].loc[date]
    vix_change.append(change)
vix_change

[-0.001696352841391124,
 0.0404290429042903,
 -0.01136363636363629,
 -0.030009680542110406,
 -0.0378378378378378,
 0.025896414342629463,
 -0.041879468845761,
 -0.04089979550102253,
 0.048489107519325454,
 -0.00563063063063051,
 -0.03165374677002585,
 -0.06677658697444341,
 -0.0045836516424752095,
 -0.05569205569205567,
 -0.03337453646477127,
 0.01501732768579132,
 0.06560846560846553,
 -0.02732644017725266,
 -0.15085536547433914,
 0.047840531561461834,
 -0.16558206796818517,
 0.04517453798767968,
 0.12255406797116387,
 0.06802721088435376,
 0.03725112395632627,
 -0.045280612244898016,
 -0.09453032104637335,
 -0.10206140983133917,
 0.10692135946415286,
 0.16860151187904962,
 0.05556535544187685,
 0.038485221674876745,
 -0.03142536475869809,
 0.042130365659777375,
 -0.044833475661827375,
 -0.02884235480047414,
 -0.0007256894049346726,
 0.0004442470013328104,
 -0.5621326616288833,
 0.04328358208955228,
 0.010875787063537364,
 -0.06827545615067689,
 0.056671818650180394,
 0.081426056338028

In [20]:
dates_FOMC.weekday

Int64Index([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2,
            4, 2, 2, 2, 1, 2, 3, 0, 1, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2,
            2, 2, 2, 2, 2],
           dtype='int64')

In [21]:
vix_change_df = pd.DataFrame(vix_change)
vix_change_wide_df = pd.DataFrame(vix_change_wide)
display(vix_change_df)#.head()
vix_change_df.describe()

,0
0,-0.001696
1,0.040429
2,-0.011364
3,-0.030010
4,-0.037838
5,0.025896
6,-0.041879
7,-0.040900
8,0.048489
9,-0.005631


,0
count,49.000000
mean,-0.005588
std,0.106991
min,-0.562133
25%,-0.040900
50%,-0.000726
75%,0.048489
max,0.168602


In [22]:
sum = (vix_change_df.iloc[:,0] + 1).cumprod()
sum_df = pd.DataFrame(sum)
sum_df.rename(columns = {0:'Cumulative return'}, inplace = True)
sum_df['Date'] = dates_FOMC
sum_df.set_index('Date', inplace = True)
sum_df

sum_wide = (vix_change_wide_df.iloc[:,0] + 1).cumprod()
sum_wide_df = pd.DataFrame(sum_wide)
sum_wide_df.rename(columns = {0:'Cumulative return'}, inplace = True)
sum_wide_df['Date'] = dates_FOMC
sum_wide_df.set_index('Date', inplace = True)
sum_wide_df#.head()

,Cumulative return
Date,
2017-02-01,1.005004
2017-03-15,1.094066
2017-05-03,1.107496
2017-06-14,1.056479
2017-07-26,0.980296
2017-09-20,1.029407
2017-11-01,1.054687
2017-12-13,0.994085
2018-01-31,1.082807


In [23]:
test2 = sum_df.sample(n=sum_df.shape[0], replace=True)
test2.describe()

,Cumulative return
count,49.000000
mean,0.732888
std,0.212461
min,0.348079
25%,0.674674
50%,0.788425
75%,0.864262
max,1.038664


In [24]:
sum_df.hvplot()*sum_wide_df.hvplot()#add legend

:Overlay
   .Curve.I  :Curve   [Date]   (Cumulative return)
   .Curve.II :Curve   [Date]   (Cumulative return)

In [25]:
sum_wide_df.hvplot()

:Curve   [Date]   (Cumulative return)

In [26]:
vix_change_df.hvplot.hist()



:Histogram   [0]   (0_count)

In [27]:
vix_change_wide_df.hvplot.hist()

:Histogram   [0]   (0_count)

In [28]:


vix_change_wide_df.hvplot.hist()*vix_change_df.hvplot.hist()


#find a way to measure how much skewness there is

:Overlay
   .Histogram.I  :Histogram   [0]   (0_count)
   .Histogram.II :Histogram   [0]   (0_count)

In [29]:
vix_wide_skew = vix_change_wide_df.skew()
print("Vix Change Wide Skew Value")
display(vix_wide_skew)

vix_change_skew =vix_change_df.skew()
print("Vix Change Skew Value")
display(vix_change_skew)

Vix Change Wide Skew Value


0   -1.699213
dtype: float64

Vix Change Skew Value


0   -2.948625
dtype: float64